# BTC/USD Live MACD + Hull (5m) Quickstart

Minimal notebook: initialize connection, subscribe, bootstrap history, start a live updating MACD + Hull chart, optional diagnostics, clean shutdown.

Steps:
1. Run each cell top → bottom.
2. The chart auto-updates (FigureWidget) or refreshes via display handle fallback.
3. Use the shutdown cell when finished.

In [30]:
import asyncio, logging, os, math, time, textwrap
from datetime import datetime
import pandas as pd
from IPython.display import display, clear_output, Markdown

from tastytrade.common.logging import setup_logging
from tastytrade.config import RedisConfigManager
from tastytrade.connections import Credentials, InfluxCredentials
from tastytrade.connections.subscription import RedisSubscriptionStore
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.messaging.processors import (
    TelegrafHTTPEventProcessor,
    RedisEventProcessor,
)
from tastytrade.config.enumerations import Channels

# Pandas display tuning for richer exploratory output
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()
setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix="dev_tastytrade",
    console=True,
    file=True,
)
logging.getLogger("asyncio").setLevel(logging.WARNING)
logging.info("Environment ready.")

2025-08-09 18:24:56 - INFO:root:62:Logging initialized - writing to ../logs/dev_tastytrade_20250809.log
2025-08-09 18:24:56 - INFO:root:32:Environment ready.
2025-08-09 18:24:56 - INFO:root:32:Environment ready.


## Setup Parameters

In [31]:
# Core symbol (weekend-active crypto)
SYMBOL = "BTC/USD:CXTALP"
CANDLE_INTERVALS = ["m", "5m", "15m", "1h", "4h", "1d"]
from datetime import datetime, timedelta, time as _time
import zoneinfo

_eastern = zoneinfo.ZoneInfo("America/New_York")
now_utc = datetime.utcnow()
# Today 09:30 Eastern -> convert to UTC
_today_eastern = datetime.now(_eastern).date()
market_open_eastern = datetime.combine(
    _today_eastern, _time(hour=9, minute=30), tzinfo=_eastern
)
BACKFILL_START = market_open_eastern.astimezone(zoneinfo.ZoneInfo("UTC"))
# If we are *before* today's 09:30 Eastern in real time (e.g., pre-market), go back one prior trading day 09:30
if now_utc < BACKFILL_START.replace(tzinfo=None):
    prev_day = _today_eastern - timedelta(days=1)
    market_open_eastern_prev = datetime.combine(
        prev_day, _time(hour=9, minute=30), tzinfo=_eastern
    )
    BACKFILL_START = market_open_eastern_prev.astimezone(zoneinfo.ZoneInfo("UTC"))

WATCH_SECONDS = 90
REFRESH_SECS = 5
FORWARD_FILL_LOOKBACK_DAYS = 3
INTERVAL = "5m"
MIN_ROWS = 6
SYMBOL_EVENT_PREFIX = SYMBOL
logging.info("Backfill start (UTC) set to %s", BACKFILL_START.isoformat())

2025-08-09 18:24:56 - INFO:root:29:Backfill start (UTC) set to 2025-08-09T13:30:00+00:00


## Connect

In [32]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize(force=True)
credentials = Credentials(config=config, env="Live")

# DXLink with Redis-backed subscription store (for replay / state)
dxlink = DXLinkManager(subscription_store=RedisSubscriptionStore())
await dxlink.open(credentials=credentials)

# Optional telemetry / distribution processors (Telegraf HTTP + Redis Pub/Sub)
for handler in dxlink.router.handler.values():
    handler.add_processor(TelegrafHTTPEventProcessor())
    handler.add_processor(RedisEventProcessor())

logging.info(
    "DXLink open; processors attached. Channels: %s", list(dxlink.router.handler.keys())
)

2025-08-09 18:24:56 - INFO:tastytrade.config.manager:174:Initialized 19 variables from .env file in Redis
2025-08-09 18:24:57 - INFO:tastytrade.connections.requests:148:Session created successfully
2025-08-09 18:24:57 - INFO:tastytrade.connections.requests:148:Session created successfully
2025-08-09 18:24:57 - INFO:tastytrade.connections.subscription:118:Redis ping response: True
2025-08-09 18:24:57 - INFO:tastytrade.connections.subscription:122:Redis version: 7.4.3
2025-08-09 18:24:57 - INFO:tastytrade.connections.subscription:123:Connected clients: 8
2025-08-09 18:24:57 - INFO:tastytrade.connections.subscription:118:Redis ping response: True
2025-08-09 18:24:57 - INFO:tastytrade.connections.subscription:122:Redis version: 7.4.3
2025-08-09 18:24:57 - INFO:tastytrade.connections.subscription:123:Connected clients: 8
2025-08-09 18:24:57 - INFO:root:14:DXLink open; processors attached. Channels: [<Channels.Control: 0>, <Channels.Quote: 7>, <Channels.Trade: 5>, <Channels.Greeks: 11>, <Cha

## Subscribe

In [33]:
# Ticker subscription
await dxlink.subscribe([SYMBOL])

# Candle subscriptions with bounded wait timeout
for iv in CANDLE_INTERVALS:
    coro = dxlink.subscribe_to_candles(
        symbol=SYMBOL, interval=iv, from_time=BACKFILL_START
    )
    await asyncio.wait_for(coro, timeout=15)

logging.info(
    "Subscribed to %s (ticker + %d candle intervals).", SYMBOL, len(CANDLE_INTERVALS)
)

2025-08-09 18:24:57 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP
2025-08-09 18:24:57 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=m}
2025-08-09 18:24:57 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=5m}
2025-08-09 18:24:57 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=15m}
2025-08-09 18:24:57 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=h}
2025-08-09 18:24:57 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=4h}
2025-08-09 18:24:57 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=d}
2025-08-09 18:24:57 - INFO:root:11:Subscribed to BTC/USD:CXTALP (ticker + 6 candle intervals).
2025-08-09 18:24:57 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=m}
2025-08-09 18:24:57 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{

## Forward Fill

In [34]:
from tastytrade.utils.time_series import forward_fill

for iv in CANDLE_INTERVALS:
    event_symbol = f"{SYMBOL}{{={iv}}}"
    logging.debug("Forward-filling %s", event_symbol)
    forward_fill(symbol=event_symbol, lookback_days=FORWARD_FILL_LOOKBACK_DAYS)
logging.info("Forward fill completed.")

2025-08-09 18:24:57 - INFO:root:7:Forward fill completed.


## Live Chart (Single Plot)

Starts one live-updating MACD + Hull chart. Re-running this cell replaces the previous chart (it cancels any existing streamer) so you always end up with a single figure. If it errors, fix the issue and re-run; no fallback plot is created to keep the notebook uncluttered.

In [35]:
# Single live MACD + Hull chart
import asyncio, logging
from IPython.display import display, Markdown, clear_output
from tastytrade.realtime import LiveMACDHullStreamer
from tastytrade.config import RedisConfigManager
from tastytrade.connections import Credentials
from tastytrade.connections.subscription import RedisSubscriptionStore
from tastytrade.connections.sockets import DXLinkManager

# Maintain a global reference so re-runs can cancel the old one
if "LIVE_STREAMER_TASK" in globals():
    task = LIVE_STREAMER_TASK
    if task and not task.done():
        task.cancel()
        try:
            _ = asyncio.get_event_loop().run_until_complete(task)
        except Exception:  # noqa: BLE001
            pass
if "LIVE_STREAMER" in globals():
    prev = LIVE_STREAMER
    # Nothing to explicitly stop; we just drop reference so GC can collect

# Ensure dxlink exists (reuse from earlier cell)
if "dxlink" not in globals() or dxlink is None or not getattr(dxlink, "open_ts", None):
    config = RedisConfigManager(env_file="/workspace/.env")
    config.initialize(force=True)
    credentials = Credentials(config=config, env="Live")
    dxlink = DXLinkManager(subscription_store=RedisSubscriptionStore())
    await dxlink.open(credentials=credentials)
    logging.info("DXLink opened (live chart cell)")

# Ensure subscriptions for just the active interval (others may already exist)
try:
    await dxlink.subscribe([SYMBOL])
    await asyncio.wait_for(
        dxlink.subscribe_to_candles(
            symbol=SYMBOL,
            interval=INTERVAL,
            from_time=BACKFILL_START,
        ),
        timeout=20,
    )
except Exception as exc:  # noqa: BLE001
    logging.warning("Subscription warning: %s", exc)

# Create streamer
streamer = LiveMACDHullStreamer(
    symbol=SYMBOL,
    interval=INTERVAL,
    min_rows=MIN_ROWS,
    poll_secs=1.0,
    auto_display=False,
)
LIVE_STREAMER = streamer

# Clear previous output area (single plot UX)
clear_output(wait=True)
print("Bootstrapping chart ...")


async def _run():
    fig_wrapper = await streamer.start(dxlink)
    display(fig_wrapper.figure)
    print("Live MACD + Hull chart running.")


LIVE_STREAMER_TASK = asyncio.create_task(_run(), name="live-macd-hull")
LIVE_STREAMER_TASK

Bootstrapping chart ...


<Task pending name='live-macd-hull' coro=<_run() running at /tmp/ipykernel_468743/1238927487.py:61>>

2025-08-09 18:26:01 - INFO:tastytrade.connections.sockets:176:Websocket listener stopped
2025-08-09 18:26:01 - INFO:tastytrade.connections.sockets:190:Keepalive stopped
2025-08-09 18:26:01 - INFO:tastytrade.connections.sockets:190:Keepalive stopped
2025-08-09 18:26:01 - INFO:tastytrade.messaging.handlers:123:Control listener stopped for channel 0
2025-08-09 18:26:01 - INFO:tastytrade.messaging.handlers:123:Control listener stopped for channel 0
2025-08-09 18:26:01 - INFO:tastytrade.messaging.handlers:130:Channel 0 metrics - Total messages: 281, Errors: 0, Max queue size: 21
2025-08-09 18:26:01 - INFO:tastytrade.messaging.handlers:130:Channel 0 metrics - Total messages: 281, Errors: 0, Max queue size: 21
2025-08-09 18:26:01 - INFO:tastytrade.messaging.handlers:123:Quote listener stopped for channel 7
2025-08-09 18:26:01 - INFO:tastytrade.messaging.handlers:123:Quote listener stopped for channel 7
2025-08-09 18:26:01 - INFO:tastytrade.messaging.handlers:130:Channel 7 metrics - Total mess

## Shutdown

In [36]:
await dxlink.close()
logging.info("DXLink closed. Notebook complete.")

2025-08-09 18:26:01 - INFO:tastytrade.connections.sockets:450:Listener task cancelled
2025-08-09 18:26:01 - INFO:tastytrade.connections.sockets:450:Keepalive task cancelled
2025-08-09 18:26:01 - INFO:tastytrade.connections.sockets:450:Keepalive task cancelled
2025-08-09 18:26:01 - INFO:tastytrade.connections.routing:58:Initiating cleanup...
2025-08-09 18:26:01 - INFO:tastytrade.connections.routing:82:Cleanup completed
2025-08-09 18:26:01 - INFO:tastytrade.connections.requests:175:Session closed
2025-08-09 18:26:01 - INFO:tastytrade.connections.sockets:444:Connection closed and cleaned up
2025-08-09 18:26:01 - INFO:tastytrade.connections.routing:58:Initiating cleanup...
2025-08-09 18:26:01 - INFO:tastytrade.connections.routing:82:Cleanup completed
2025-08-09 18:26:01 - INFO:tastytrade.connections.requests:175:Session closed
2025-08-09 18:26:01 - INFO:tastytrade.connections.sockets:444:Connection closed and cleaned up
2025-08-09 18:26:01 - INFO:root:2:DXLink closed. Notebook complete.
20